In [87]:
# Cretea local spark cluster
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [88]:
# Q1.Execute spark.version. What's the output?
# A 1.3.3.2
spark.version

'3.3.2'

In [ ]:
# Q2.What is the average size of the Parquet 

df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-06.csv')
schema = types.StructType([
 types.StructField('dispatching_base_num', types.StringType(), True),
 types.StructField('pickup_datetime', types.TimestampType(), True),
 types.StructField('dropoff_datetime', types.TimestampType(), True),
 types.StructField('PULocationID', types.IntegerType(), True),
 types.StructField('DOLocationID', types.IntegerType(), True),
 types.StructField('SR_Flag', types.StringType(), True),
 types.StructField('Affiliated_base_number', types.StringType(), True)])

df = spark.read \
.option("header", "true") \
.schema(schema) \
.csv('fhvhv_tripdata_2021-06.csv')
df = df.repartition(12)
df.write.parquet('fhvhv/2021/06/',)

# Execting !ls -lh fhvhv/2021/06/ in the terminal
# A.2. 24MB

In [89]:
# Q3 How many taxi trips were started on June 15th?
from pyspark.sql import functions as F
df = spark.read.parquet('fhvhv/2021/06/')
# Change datetime to date in a new column
df = df \
     .withColumn('pickup_date', F.to_date(df.pickup_datetime))

# A 3.452470
df.filter(df.pickup_date == '2021-06-15').count()
    

452470

In [90]:
# Q4 How long is the longest trip in the dataset? 
df_time = df \
    .select('dropoff_datetime','pickup_datetime') \
        .withColumn('difsecond',(df.dropoff_datetime-df.pickup_datetime).cast('long')/3600)

#A. 1.66.87
df_time.select(F.max(df_time.difsecond)).show()

+----------------+
|  max(difsecond)|
+----------------+
|66.8788888888889|
+----------------+



In [91]:
# Q5 What port does Spark's User Interface Dashboard run on by default?
# After start the local instance can access the interface with http://localhost:4040/
# A.3.4040

In [92]:
# Q6 What is the name of the most frequent pickup location zone?
schema_zone = types.StructType([
 types.StructField('LocationID', types.IntegerType(), True),
 types.StructField('Borough', types.StringType(), True),
 types.StructField('Zone', types.StringType(), True),
 types.StructField('service_zone', types.StringType(), True)])

df_location = spark.read.csv('taxi+_zone_lookup.csv',header=True,schema=schema_zone)
df_join = df.join(df_location,df.PULocationID == df_location.LocationID,'inner')

# A. 4.Crown Heights North
zone_max = df_join \
    .groupBy('Zone') \
    .agg(F.count('Zone').alias('count_location')) \
    .sort(F.desc('count_location')).show(1)

+-------------------+--------------+
|               Zone|count_location|
+-------------------+--------------+
|Crown Heights North|        231279|
+-------------------+--------------+
only showing top 1 row

